<a href="https://colab.research.google.com/github/Estampille/Cognitive-Services-Vision-Solution-Templates/blob/master/googlesentiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:


!pip install --upgrade google-api-python-client
!pip install google-cloud-language
!pip3 install --upgrade google-cloud-storage
!pip install --upgrade pip

In [3]:
!pip install word2number

!pip install contractions

In [107]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import argparse
from google.cloud import language_v1


attribution sheet

In [128]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

sh = gc.open('COUNTERPOINT_extrait_tweet')


shfr = sh.worksheet('climate fr') 





pre-processing data

In [129]:
from bs4 import BeautifulSoup
import spacy
import re
from word2number import w2n
import contractions

In [137]:
shfr_html=[]
for i in range(20):
  text = text = "{}".format([shfr.cell(i+2, 4).value])
  def strip_html_tags(text):
      """remove html tags from text"""
      linkr = re.sub(r"http\S+", "", text)
      hashr=re.sub(r"#","",linkr)
      result=hashr.replace("\\n", " ").replace("\\","").replace("-", " ").replace("@", "").replace("\"","").replace("é","e").replace("è","e").replace("'", " ").replace("’", " ")
            
      soup = BeautifulSoup(result, "html.parser")
      
      stripped_text = soup.get_text(separator=" ")
      return stripped_text
  shfr_html.append("{}".format(strip_html_tags(text)))



In [146]:

print(shfr_html[7])




[ Il reste 131 jours avant la fin de l annee. Et pourtant, nous avons consomme toutes les ressources que la planete peut generer en 1 an. C est le JourDuDepassement. Continuons d agir ! 30 milliards d euros du plan de relance seront consacres à la transition ecologique. ]


sentiment analyze

In [139]:
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file('apikey.json')
client = language.LanguageServiceClient(credentials=credentials)
for i in range (10):
  
  def tweet(client):

    results=[]
    text = "{}".format(shfr_html[i])
    #Setting the Service Account key
    
  
    document = types.Document(
        content=text,
        type=enums.Document.Type.PLAIN_TEXT)
    sentiment = client.analyze_sentiment(document=document).document_sentiment

    


    if sentiment.score <0 :
      results.append("négatif")
    elif sentiment.score >0:
      results.append("positif")
    else:
      results.append("mixe ou incertain")

    results.append('Text: {}     '.format(text))
    
    results.append('   Sentiment: {}, Magnitude{}'.format(sentiment.score, sentiment.magnitude))
    print(result)
    print(sentiment)
    print (client.analyze_sentiment(document=document))
    return results
    

  result= tweet(client)
  shfr.update_cell( i+2,11, ' '.join(result))

['positif', 'Text: [🇺🇸 FLASH   Joe Biden annonce que s il est elu, il rejoindra l accord de Paris sur le climat des le premier jour de sa presidence. (AFP)]     ', '   Sentiment: 0.10000000149011612, Magnitude0.20000000298023224']
magnitude: 0.30000001192092896
score: -0.30000001192092896

document_sentiment {
  magnitude: 0.30000001192092896
  score: -0.30000001192092896
}
language: "fr"
sentences {
  text {
    content: "[ le rechauffement climatique, la menace d une 3e guerre ptdr les etudes pourquoi faire ? ]"
    begin_offset: -1
  }
  sentiment {
    magnitude: 0.30000001192092896
    score: -0.30000001192092896
  }
}

['négatif', 'Text: [ le rechauffement climatique, la menace d une 3e guerre ptdr les etudes pourquoi faire ? ]     ', '   Sentiment: -0.30000001192092896, Magnitude0.30000001192092896']

document_sentiment {
}
language: "fr"
sentences {
  text {
    content: "[L empreinte carbone de ton dropshipping on peut en parler aussi ?]"
    begin_offset: -1
  }
  sentiment {

In [ ]:
print(results)